# Regionalização de Bacias usando Redes Neurais

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# %%bash
# cp -r /content/drive/"My Drive"/DEHA/RegBaciasCE /content
# mkdir -p RegBaciasCE/src/models/NewFFNN3/CrossValid
# nvidia-smi

## Importing Packages

In [2]:
import sys
sys.path.append("..")
sys.path.append("../src")
sys.path.append("../scripts")
sys.path.append("../database")

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
#from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from funcoes_auxiliares import *
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.utils import shuffle
import time
import matplotlib.pyplot as plt


In [4]:
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam, Nadam, RMSprop

In [5]:
pd.options.display.max_columns = 999

import warnings

warnings.filterwarnings('ignore')


In [6]:
bacias_com_problema = [36220000]

In [7]:
dados = pd.read_csv("../database/Formated inputs/NewFFNN2Data.csv").query(f"`Estações ANA` not in {bacias_com_problema}")\
                                .reset_index(drop=True)
bacias = list(set(dados['Estações ANA']))
bacias.sort()
print(len(dados))
dados.head()

9164


,Estações ANA,Declividade - D (%),Comprimento do rio principal - CT (km),Área de contribuição da estação fluviométrica - A (km2),Perímetro da área de contribuição da estação fluviométrica - P (km),Comprimento total de drenagem - CTD (km),Densidade de drenagem - DD (km-1),Curve number médio – CN (mm),Parcela da bacia no cristalino - Cr,E_0,P_2,P_1,P_0,Q_2,Q_1,Q_0
0,34730000,0.069435,64319.016213,897.371509,211.282396,528.790311,0.589266,56.773914,0.0,121.889904,204.2900,135.78000,142.92000,7.884968,6.138600,3.870129
1,34730000,0.069435,64319.016213,897.371509,211.282396,528.790311,0.589266,56.773914,0.0,129.330288,135.7800,142.92000,12.50800,6.138600,3.870129,2.136953
2,34730000,0.069435,64319.016213,897.371509,211.282396,528.790311,0.589266,56.773914,0.0,151.559616,142.9200,12.50800,3.03470,3.870129,2.136953,1.040503
3,34730000,0.069435,64319.016213,897.371509,211.282396,528.790311,0.589266,56.773914,0.0,174.559616,12.5080,3.03470,0.96898,2.136953,1.040503,0.629442
4,34730000,0.069435,64319.016213,897.371509,211.282396,528.790311,0.589266,56.773914,0.0,187.559616,3.0347,0.96898,1.95120,1.040503,0.629442,0.388273


In [8]:
drop_columns = ['Estações ANA',
       'Declividade - D (%)',
       'Comprimento do rio principal - CT (km)',
       'Comprimento total de drenagem - CTD (km)',
       'Perímetro da área de contribuição da estação fluviométrica - P (km)',
       'Densidade de drenagem - DD (km-1)',
       'E_2', 'E_1', 'Q_2', 'Q_1', 
       'Q_0']


columns = [
       'Área de contribuição da estação fluviométrica - A (km2)',
       'Comprimento total de drenagem - CTD (km)',
       'Curve number médio – CN (mm)', 'Parcela da bacia no cristalino - Cr',
       'E_0', 'P_2', 'P_1', 'P_0',
        'Q_2','Q_1'
#        'E_0', 'P_2', 'P_1', 'P_0',
#         'Q_2','Q_1'
]

In [9]:
#X = dados.drop(drop_columns,axis=1)
X = dados[columns]
Y = dados['Q_0'].values

#scaler = StandardScaler()
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X,columns=columns)

In [10]:
print(len(X))
X.head()

9164


,Área de contribuição da estação fluviométrica - A (km2),Comprimento total de drenagem - CTD (km),Curve number médio – CN (mm),Parcela da bacia no cristalino - Cr,E_0,P_2,P_1,P_0,Q_2,Q_1
0,0.019908,0.011969,0.0,0.0,0.377420,0.333448,0.221624,0.233278,0.006548,0.005097
1,0.019908,0.011969,0.0,0.0,0.432495,0.221624,0.233278,0.020416,0.005097,0.003214
2,0.019908,0.011969,0.0,0.0,0.597038,0.233278,0.020416,0.004953,0.003214,0.001774
3,0.019908,0.011969,0.0,0.0,0.767287,0.020416,0.004953,0.001582,0.001774,0.000864
4,0.019908,0.011969,0.0,0.0,0.863514,0.004953,0.001582,0.003185,0.000864,0.000523


### Definição do modelo

In [11]:
# Parâmetros
INIT_LR = 0.001
EPOCHS = 250
BS = 64  # Diminua o tamanho dos lotes se você não tiver memória suficiente
SEEDS = [1500, 4, 30, 6000, 400, 648, 900,10000, 2000, 5432]
SEED=SEEDS[3]
print("SEED: %s" % SEED)
n_input = X.shape[1]
n_classes = 1

NUM_HIDDN = 50

SEED: 6000


In [12]:
# Defining the model
model = tfMLP().build_model(n_input,1,2,'relu',NUM_HIDDN,SEED=SEED)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 main_input (InputLayer)     [(None, 10)]              0         
                                                                 
 hidden_layer0 (Dense)       (None, 50)                550       
                                                                 
 hidden_layer1 (Dense)       (None, 50)                2550      
                                                                 
 streamflow (Dense)          (None, 1)                 51        
                                                                 
Total params: 3,151
Trainable params: 3,151
Non-trainable params: 0
_________________________________________________________________


2022-02-25 10:23:37.298164: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# Training parameters (used only as parameters to model.compile)

opt = RMSprop(learning_rate=INIT_LR)

metrics = NSE_K
loss = RMSE_Keras

relu = lambda x: max(x,0)

In [14]:
# Models weights directory
model_dir = '../src/models/NewFFNN2/CrossValid_SEED_7173_NSE_0824/'
model_code = '2'


In [15]:
nome, n_obs,nse, cor, rmse = [], [], [], [], []

for bacia in bacias:
    idx = dados['Estações ANA'] == bacia

    X_treino, Y_treino = X[~idx], Y[~idx]
    X_teste, Y_teste = X[idx], Y[idx]

    
    model.load_weights(os.path.join(model_dir,'weights.NewFFN%s_%d.hdf5'%(model_code,bacia)))
    model.compile(loss=[loss],optimizer = opt,
                metrics=[metrics])


    Y_ = model.predict(X_teste)

    Y_Prev = np.vectorize(relu)(Y_)


    Y_test = np.expand_dims(Y_teste,1)

    k_ = np.mean(Y_test) / np.mean(Y_Prev)

    nome.append(bacia)
    cor.append(np.corrcoef(Y_Prev.T,Y_test.T)[1][0])
    rmse.append(RMSE(Y_test,Y_Prev))
    nse.append(NASH(Y_test,Y_Prev))
    n_obs.append(len(Y_teste))


pd.DataFrame({"Bacia":nome, "Observacoes":n_obs, "NSE":nse,"Cor": cor,"RMSE": rmse})

,Bacia,Observacoes,NSE,Cor,RMSE
0,34730000,443,-3.558889,0.706104,10.724443
1,34740000,70,0.505180,0.876498,4.922226
2,34750000,508,0.882741,0.942915,33.170541
3,35050000,95,0.541874,0.823920,5.961048
4,35125000,309,0.915184,0.960450,5.673409
5,35170000,443,0.894877,0.952180,19.539618
6,35210000,570,0.925421,0.965254,6.833693
7,35223000,124,0.278338,0.593697,65.937380
8,35240000,392,0.739142,0.877760,9.063669
9,35260000,494,0.604040,0.914012,17.207909


### Performance dos modelos

In [16]:
results = pd.DataFrame({"Bacia":nome, "Observacoes":n_obs, "NSE":nse,"Cor": cor,"RMSE": rmse})


In [17]:
results["NSE"].apply(relu).mean()#[results.Bacia != 36220000]


0.7132066407101485

In [18]:
results[results["Bacia"] != 35668000]["NSE"].apply(relu).mean()
results.query(f"""Bacia not in {bacias_com_problema+[35668000]}""")["NSE"].apply(relu).mean()

0.7442156250888506

In [19]:
bacias_barros_2013 = (34750000, 35050000, 35125000, 35170000, 35210000,
                      35260000, 35263000, 35880000, 35950000, 36020000,
                      36125000, 36130000, 36160000, 36210000, 36250000,
                      36270000, 36290000, 36520000)
results_barros_2013 = results.query(f"""Bacia in {bacias_barros_2013}
""")["NSE"].apply(relu).values.mean()


results_barros_2013

0.8237213748281907

In [20]:
results.query(f"""Bacia in {bacias_barros_2013}
""")["NSE"].apply(relu).median()

0.8661113706545847

In [22]:
results.to_csv("../Database/output/ResultadosFFNN2.csv",index=False)